In [2]:
import numpy as np
import pandas as pd
import novosparc
from spaotsc import SpaOTsc
import tangram as tg

import anndata as ad
import scanpy as sc

from sklearn.model_selection import KFold

import os
import time

from SC2Spa import SI, PP, Vis

/mnt/win1
/mnt/win1/SC2Spa_Notebook/MOB_Visium


In [3]:
ST = pd.read_csv('MOBRep12_Processed.csv')
loc = pd.read_csv('MOBRep12_loc.csv')[['x', 'y']]

In [23]:
ST.sum(axis = 1)

0       9999.999458
1      10000.000044
2       9999.999678
3       9999.999738
4      10000.000211
           ...     
273    10000.000208
274    10000.000035
275     9999.999581
276    10000.000401
277     9999.999814
Length: 278, dtype: float64

In [4]:
print(ST.shape)
ST.head()

(278, 14151)


,Plekha1,Cnih3,Gpc5,Sirt3,Rab8a,Nadk,Tiparp,Fbxo28,Rgl1,Ctsb,...,Agr2,Il12a,Dhx37,Rnase1,2310034G01Rik,Pdzk1ip1,Gipr,Dpys,Hapln3,Snora21
0,2.982759,0.596552,0.596552,1.193104,2.982759,0.596552,1.193104,1.789656,0.596552,4.175863,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.042753,1.042753,0.000000,1.042753,1.042753,1.042753,0.000000,1.042753,0.000000,9.384776,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.999800,0.000000,0.000000,0.000000,0.000000,0.999800,0.999800,1.999600,3.999200,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.000000,0.000000,0.000000,1.490313,0.000000,0.000000,2.980626,7.451565,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.770832,0.000000,0.000000,0.000000,0.770832,0.770832,0.000000,0.770832,0.000000,7.708317,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
print(loc.shape)
loc.head()

(278, 2)


,x,y
0,16.918,16.996
1,18.017,17.034
2,20.075,17.059
3,18.979,17.065
4,21.937,16.967


In [6]:
loc_Norm = PP.MinMaxNorm(loc, Y_ref = loc)
print(loc_Norm.shape)
loc_Norm.head()

(278, 2)


,x,y
0,0.407109,0.200919
1,0.457066,0.203450
2,0.550616,0.205115
3,0.500795,0.205514
4,0.635256,0.198988


In [7]:
adata = ad.AnnData(X=ST.values, var = pd.DataFrame(ST.columns).set_index(0))
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

X = adata.X
Y = loc_Norm.values

# Split groups

In [8]:
'''
root = 'CV_groups/'
if not os.path.exists(root):
    os.makedirs(root)
    
kf = KFold(n_splits=5, random_state = 2022, shuffle = True)
cv_index = kf.split(X)

i = 0
for cv_train, cv_test in cv_index:
    i+=1
    pd.DataFrame(cv_train).to_csv(root + 'index_train_' + str(i) + '.csv', header = None)
    pd.DataFrame(cv_test).to_csv(root + 'index_test_' + str(i) + '.csv', header = None)

'''

train_indices, test_indices = PP.LoadIndices()


# SC2Spa

In [14]:
sta = time.time()

history, train_preds, test_preds = SI.CrossValidation(X=X, Y=Y,
                                          train_indices = train_indices, test_indices = test_indices,
                                          l1_reg = 1e-5, l2_reg = 1e-4, dropout = 0.05, epoch = 500,
                                          batch_size = 16,
                                          nodes = [4096, 1024, 256, 64, 16, 4], seed = None)

end = time.time()
print((end - sta) / 60.0, 'min')

0
Epoch 1/500
13/13 [==============================] - 4s 59ms/step - loss: 6.2334 - rmse: 0.2591 - val_loss: 6.1145 - val_rmse: 0.5311 - lr: 0.0010
Epoch 2/500
13/13 [==============================] - 0s 32ms/step - loss: 5.6336 - rmse: 0.2317 - val_loss: 5.7036 - val_rmse: 0.5234 - lr: 0.0010
Epoch 3/500
13/13 [==============================] - 0s 33ms/step - loss: 5.1619 - rmse: 0.2174 - val_loss: 5.0708 - val_rmse: 0.4107 - lr: 0.0010
Epoch 4/500
13/13 [==============================] - 0s 32ms/step - loss: 4.6286 - rmse: 0.2063 - val_loss: 4.5850 - val_rmse: 0.4367 - lr: 0.0010
Epoch 5/500
13/13 [==============================] - 0s 31ms/step - loss: 4.1322 - rmse: 0.2032 - val_loss: 4.1107 - val_rmse: 0.4264 - lr: 0.0010
Epoch 6/500
13/13 [==============================] - 0s 32ms/step - loss: 3.6892 - rmse: 0.1915 - val_loss: 3.5567 - val_rmse: 0.2670 - lr: 0.0010
Epoch 7/500
13/13 [==============================] - 0s 32ms/step - loss: 3.3102 - rmse: 0.1838 - val_loss: 3.2653 -

Epoch 55/500
13/13 [==============================] - 0s 33ms/step - loss: 0.7050 - rmse: 0.1233 - val_loss: 0.8891 - val_rmse: 0.2852 - lr: 5.0000e-04
Epoch 56/500
13/13 [==============================] - 0s 32ms/step - loss: 0.7374 - rmse: 0.1233 - val_loss: 0.8842 - val_rmse: 0.2703 - lr: 5.0000e-04
Epoch 57/500
13/13 [==============================] - 0s 31ms/step - loss: 0.7351 - rmse: 0.1246 - val_loss: 0.8669 - val_rmse: 0.2721 - lr: 5.0000e-04
Epoch 58/500
13/13 [===========================>..] - ETA: 0s - loss: 0.6890 - rmse: 0.1164
Epoch 58: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
13/13 [==============================] - 0s 32ms/step - loss: 0.6891 - rmse: 0.1183 - val_loss: 0.7910 - val_rmse: 0.2494 - lr: 5.0000e-04
Epoch 59/500
13/13 [==============================] - 0s 32ms/step - loss: 0.6299 - rmse: 0.1104 - val_loss: 0.7438 - val_rmse: 0.2525 - lr: 2.5000e-04
Epoch 60/500
13/13 [==============================] - 0s 32ms/step - loss: 0.5654 - 

Epoch 107/500
13/13 [==============================] - 0s 32ms/step - loss: 0.2639 - rmse: 0.0814 - val_loss: 0.4281 - val_rmse: 0.2488 - lr: 1.2500e-04
Epoch 108/500
13/13 [==============================] - 0s 32ms/step - loss: 0.2674 - rmse: 0.0896 - val_loss: 0.4357 - val_rmse: 0.2586 - lr: 1.2500e-04
Epoch 109/500
13/13 [==============================] - 0s 32ms/step - loss: 0.2575 - rmse: 0.0806 - val_loss: 0.4419 - val_rmse: 0.2656 - lr: 1.2500e-04
Epoch 110/500
13/13 [==============================] - 0s 32ms/step - loss: 0.2550 - rmse: 0.0797 - val_loss: 0.4416 - val_rmse: 0.2684 - lr: 1.2500e-04
Epoch 111/500
13/13 [==============================] - 0s 31ms/step - loss: 0.2592 - rmse: 0.0879 - val_loss: 0.4329 - val_rmse: 0.2637 - lr: 1.2500e-04
Epoch 112/500
13/13 [==============================] - 0s 32ms/step - loss: 0.2523 - rmse: 0.0830 - val_loss: 0.4380 - val_rmse: 0.2686 - lr: 1.2500e-04
Epoch 113/500
13/13 [==============================] - 0s 33ms/step - loss: 0.2524

13/13 [==============================] - 0s 32ms/step - loss: 0.1467 - rmse: 0.0685 - val_loss: 0.3158 - val_rmse: 0.2378 - lr: 7.8125e-06
Epoch 210/500
13/13 [==============================] - 0s 31ms/step - loss: 0.1406 - rmse: 0.0628 - val_loss: 0.3161 - val_rmse: 0.2385 - lr: 7.8125e-06
Epoch 211/500
13/13 [==============================] - 0s 33ms/step - loss: 0.1515 - rmse: 0.0741 - val_loss: 0.3162 - val_rmse: 0.2390 - lr: 7.8125e-06
Epoch 212/500
13/13 [==============================] - 0s 32ms/step - loss: 0.1529 - rmse: 0.0757 - val_loss: 0.3170 - val_rmse: 0.2399 - lr: 7.8125e-06
Epoch 213/500
13/13 [==============================] - 0s 33ms/step - loss: 0.1498 - rmse: 0.0727 - val_loss: 0.3142 - val_rmse: 0.2369 - lr: 7.8125e-06
Epoch 214/500
13/13 [==============================] - 0s 32ms/step - loss: 0.1499 - rmse: 0.0725 - val_loss: 0.3136 - val_rmse: 0.2363 - lr: 7.8125e-06
Epoch 215/500
13/13 [==============================] - 0s 32ms/step - loss: 0.1460 - rmse: 0.068

13/13 [==============================] - 0s 34ms/step - loss: 0.1350 - rmse: 0.0671 - val_loss: 0.3010 - val_rmse: 0.2332 - lr: 2.4414e-07
Epoch 311/500
13/13 [==============================] - 0s 32ms/step - loss: 0.1318 - rmse: 0.0640 - val_loss: 0.3013 - val_rmse: 0.2335 - lr: 2.4414e-07
Epoch 312/500
13/13 [==============================] - 0s 33ms/step - loss: 0.1392 - rmse: 0.0713 - val_loss: 0.3012 - val_rmse: 0.2334 - lr: 2.4414e-07
Epoch 313/500
13/13 [==============================] - 0s 33ms/step - loss: 0.1445 - rmse: 0.0767 - val_loss: 0.3013 - val_rmse: 0.2335 - lr: 2.4414e-07
Epoch 314/500
13/13 [==============================] - 0s 32ms/step - loss: 0.1290 - rmse: 0.0612 - val_loss: 0.3013 - val_rmse: 0.2335 - lr: 2.4414e-07
Epoch 315/500
13/13 [==============================] - 0s 33ms/step - loss: 0.1327 - rmse: 0.0649 - val_loss: 0.3014 - val_rmse: 0.2336 - lr: 2.4414e-07
Epoch 316/500
13/13 [==============================] - 0s 33ms/step - loss: 0.1495 - rmse: 0.081

...

13/13 [==============================] - 0s 31ms/step - loss: 5.2050 - rmse: 0.2130 - val_loss: 5.3502 - val_rmse: 0.5181 - lr: 0.0010
Epoch 6/500
13/13 [==============================] - 0s 33ms/step - loss: 4.9045 - rmse: 0.2047 - val_loss: 5.0517 - val_rmse: 0.5035 - lr: 0.0010
Epoch 7/500
13/13 [==============================] - 0s 33ms/step - loss: 4.6292 - rmse: 0.2049 - val_loss: 4.7585 - val_rmse: 0.4747 - lr: 0.0010
Epoch 8/500
13/13 [==============================] - 0s 33ms/step - loss: 4.3663 - rmse: 0.1977 - val_loss: 4.4607 - val_rmse: 0.4193 - lr: 0.0010
Epoch 9/500
13/13 [==============================] - 0s 33ms/step - loss: 4.1238 - rmse: 0.1879 - val_loss: 4.0688 - val_rmse: 0.2533 - lr: 0.0010
Epoch 10/500
13/13 [==============================] - 0s 31ms/step - loss: 3.8891 - rmse: 0.1774 - val_loss: 3.8464 - val_rmse: 0.2539 - lr: 0.0010
Epoch 11/500
13/13 [==============================] - 0s 32ms/step - loss: 3.6662 - rmse: 0.1726 - val_loss: 3.6381 - val_rmse: 0

Epoch 59/500
13/13 [==============================] - 0s 32ms/step - loss: 0.7400 - rmse: 0.1192 - val_loss: 0.8535 - val_rmse: 0.2334 - lr: 5.0000e-04
Epoch 60/500
13/13 [==============================] - 0s 33ms/step - loss: 0.7478 - rmse: 0.1183 - val_loss: 0.9218 - val_rmse: 0.2811 - lr: 5.0000e-04
Epoch 61/500
13/13 [==============================] - 0s 32ms/step - loss: 0.7737 - rmse: 0.1195 - val_loss: 0.9067 - val_rmse: 0.2450 - lr: 5.0000e-04
Epoch 62/500
13/13 [==============================] - 0s 32ms/step - loss: 0.7738 - rmse: 0.1169 - val_loss: 0.9007 - val_rmse: 0.2520 - lr: 5.0000e-04
Epoch 63/500
13/13 [==============================] - 0s 32ms/step - loss: 0.7478 - rmse: 0.1084 - val_loss: 0.8758 - val_rmse: 0.2506 - lr: 5.0000e-04
Epoch 64/500
13/13 [==============================] - 0s 32ms/step - loss: 0.7195 - rmse: 0.1069 - val_loss: 0.8241 - val_rmse: 0.2314 - lr: 5.0000e-04
Epoch 65/500
13/13 [==============================] - 0s 32ms/step - loss: 0.6889 - rmse

Epoch 111/500
13/13 [==============================] - 0s 33ms/step - loss: 0.3626 - rmse: 0.0904 - val_loss: 0.5241 - val_rmse: 0.2556 - lr: 1.2500e-04
Epoch 112/500
13/13 [==============================] - 0s 32ms/step - loss: 0.3596 - rmse: 0.0942 - val_loss: 0.5098 - val_rmse: 0.2476 - lr: 1.2500e-04
Epoch 113/500
13/13 [==============================] - 0s 33ms/step - loss: 0.3459 - rmse: 0.0847 - val_loss: 0.4977 - val_rmse: 0.2382 - lr: 1.2500e-04
Epoch 114/500
13/13 [==============================] - 0s 33ms/step - loss: 0.3366 - rmse: 0.0799 - val_loss: 0.4916 - val_rmse: 0.2370 - lr: 1.2500e-04
Epoch 115/500
13/13 [==============================] - 0s 33ms/step - loss: 0.3515 - rmse: 0.0977 - val_loss: 0.4985 - val_rmse: 0.2442 - lr: 1.2500e-04
Epoch 116/500
13/13 [==============================] - 0s 33ms/step - loss: 0.3443 - rmse: 0.0876 - val_loss: 0.5097 - val_rmse: 0.2519 - lr: 1.2500e-04
Epoch 117/500
13/13 [==============================] - 0s 32ms/step - loss: 0.3516

Epoch 215/500
13/13 [==============================] - 0s 32ms/step - loss: 0.2207 - rmse: 0.0738 - val_loss: 0.3797 - val_rmse: 0.2334 - lr: 1.5625e-05
Epoch 216/500
13/13 [==============================] - 0s 33ms/step - loss: 0.2280 - rmse: 0.0820 - val_loss: 0.3775 - val_rmse: 0.2319 - lr: 1.5625e-05
Epoch 217/500
13/13 [==============================] - 0s 32ms/step - loss: 0.2253 - rmse: 0.0799 - val_loss: 0.3799 - val_rmse: 0.2349 - lr: 1.5625e-05
Epoch 218/500
13/13 [==========================>...] - ETA: 0s - loss: 0.2380 - rmse: 0.0930
Epoch 218: ReduceLROnPlateau reducing learning rate to 7.812500371073838e-06.
13/13 [==============================] - 0s 33ms/step - loss: 0.2354 - rmse: 0.0905 - val_loss: 0.3821 - val_rmse: 0.2373 - lr: 1.5625e-05
Epoch 219/500
13/13 [==============================] - 0s 32ms/step - loss: 0.2180 - rmse: 0.0736 - val_loss: 0.3787 - val_rmse: 0.2349 - lr: 7.8125e-06
Epoch 220/500
13/13 [==============================] - 0s 32ms/step - loss: 0.

13/13 [==============================] - 0s 33ms/step - loss: 0.1929 - rmse: 0.0706 - val_loss: 0.3528 - val_rmse: 0.2305 - lr: 4.8828e-07
Epoch 317/500
13/13 [==============================] - 0s 32ms/step - loss: 0.1913 - rmse: 0.0691 - val_loss: 0.3531 - val_rmse: 0.2309 - lr: 4.8828e-07
Epoch 318/500
13/13 [==========================>...] - ETA: 0s - loss: 0.2096 - rmse: 0.0873
Epoch 318: ReduceLROnPlateau reducing learning rate to 2.4414063659605745e-07.
13/13 [==============================] - 0s 33ms/step - loss: 0.2066 - rmse: 0.0843 - val_loss: 0.3531 - val_rmse: 0.2309 - lr: 4.8828e-07
Epoch 319/500
13/13 [==============================] - 0s 33ms/step - loss: 0.1890 - rmse: 0.0669 - val_loss: 0.3530 - val_rmse: 0.2308 - lr: 2.4414e-07
Epoch 320/500
13/13 [==============================] - 0s 32ms/step - loss: 0.1935 - rmse: 0.0713 - val_loss: 0.3529 - val_rmse: 0.2308 - lr: 2.4414e-07
Epoch 321/500
13/13 [==============================] - 0s 33ms/step - loss: 0.1946 - rmse: 

In [16]:
root = 'SI_Benchmarking/SC2Spa/'
if not os.path.exists(root):
    os.makedirs(root)
    
for i in range(5):
    pd.DataFrame(PP.ReMMNorm(loc.values, test_preds[i])).to_csv('SI_Benchmarking/SC2Spa/pred_' + str(i) + '.csv', header = None)

# LR

In [14]:
sta = time.time()

history, train_preds, test_preds = SI.CV_LR(X=X, Y=Y,
                                         train_indices = train_indices, 
                                         test_indices = test_indices, l1_reg=1e-5, l2_reg = 0,
                                         dropout = 0.05, epoch = 500, batch_size = 16)

end = time.time()
print((end - sta) / 60.0, 'min')

0
Epoch 1/500


/mnt/win1/SC2Spa/SI.py:552: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(generator = batch_generator(X,\


13/13 [==============================] - 1s 19ms/step - loss: 0.5324 - rmse: 0.5297 - val_loss: 0.5338 - val_rmse: 0.5312 - lr: 0.0010
Epoch 2/500
13/13 [==============================] - 0s 10ms/step - loss: 0.5376 - rmse: 0.5350 - val_loss: 0.5338 - val_rmse: 0.5312 - lr: 0.0010
Epoch 3/500
13/13 [==============================] - 0s 10ms/step - loss: 0.5376 - rmse: 0.5350 - val_loss: 0.5337 - val_rmse: 0.5312 - lr: 0.0010
Epoch 4/500
10/13 [====================>.........] - ETA: 0s - loss: 0.5343 - rmse: 0.5318
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
13/13 [==============================] - 0s 10ms/step - loss: 0.5375 - rmse: 0.5350 - val_loss: 0.5337 - val_rmse: 0.5312 - lr: 0.0010
Epoch 5/500
13/13 [==============================] - 0s 10ms/step - loss: 0.5375 - rmse: 0.5350 - val_loss: 0.5336 - val_rmse: 0.5312 - lr: 5.0000e-04
Epoch 6/500
13/13 [==============================] - 0s 10ms/step - loss: 0.5374 - rmse: 0.5350 - val_loss: 0.5336 - v

Epoch 42/500
13/13 [==============================] - 0s 11ms/step - loss: 0.5373 - rmse: 0.5350 - val_loss: 0.5335 - val_rmse: 0.5312 - lr: 1.2207e-07
Epoch 43/500
10/13 [====================>.........] - ETA: 0s - loss: 0.5477 - rmse: 0.5454
Epoch 43: ReduceLROnPlateau reducing learning rate to 6.103515914901436e-08.
13/13 [==============================] - 0s 10ms/step - loss: 0.5373 - rmse: 0.5350 - val_loss: 0.5335 - val_rmse: 0.5312 - lr: 1.2207e-07
Epoch 44/500
13/13 [==============================] - 0s 11ms/step - loss: 0.5373 - rmse: 0.5350 - val_loss: 0.5335 - val_rmse: 0.5312 - lr: 6.1035e-08
Epoch 45/500
13/13 [==============================] - 0s 11ms/step - loss: 0.5373 - rmse: 0.5350 - val_loss: 0.5335 - val_rmse: 0.5312 - lr: 6.1035e-08
Epoch 46/500
 9/13 [==================>...........] - ETA: 0s - loss: 0.5206 - rmse: 0.5183
Epoch 46: ReduceLROnPlateau reducing learning rate to 3.051757957450718e-08.
13/13 [==============================] - 0s 11ms/step - loss: 0.537

Epoch 92/500
13/13 [==============================] - 0s 10ms/step - loss: 0.5373 - rmse: 0.5350 - val_loss: 0.5335 - val_rmse: 0.5312 - lr: 1.0000e-08
Epoch 93/500
13/13 [==============================] - 0s 10ms/step - loss: 0.5373 - rmse: 0.5350 - val_loss: 0.5335 - val_rmse: 0.5312 - lr: 1.0000e-08
Epoch 94/500
13/13 [==============================] - 0s 11ms/step - loss: 0.5373 - rmse: 0.5350 - val_loss: 0.5335 - val_rmse: 0.5312 - lr: 1.0000e-08
Epoch 95/500
13/13 [==============================] - 0s 10ms/step - loss: 0.5373 - rmse: 0.5350 - val_loss: 0.5335 - val_rmse: 0.5312 - lr: 1.0000e-08
Epoch 96/500
13/13 [==============================] - 0s 10ms/step - loss: 0.5373 - rmse: 0.5350 - val_loss: 0.5335 - val_rmse: 0.5312 - lr: 1.0000e-08
Epoch 97/500
13/13 [==============================] - 0s 9ms/step - loss: 0.5373 - rmse: 0.5350 - val_loss: 0.5335 - val_rmse: 0.5312 - lr: 1.0000e-08
Epoch 98/500
13/13 [==============================] - 0s 10ms/step - loss: 0.5373 - rmse:

...

Epoch 18/500
13/13 [==============================] - 0s 9ms/step - loss: 0.0493 - rmse: 0.0463 - val_loss: 0.2445 - val_rmse: 0.2415 - lr: 1.2500e-04
Epoch 19/500
10/13 [====================>.........] - ETA: 0s - loss: 0.0545 - rmse: 0.0515
Epoch 19: ReduceLROnPlateau reducing learning rate to 6.25000029685907e-05.
13/13 [==============================] - 0s 10ms/step - loss: 0.0511 - rmse: 0.0481 - val_loss: 0.2444 - val_rmse: 0.2414 - lr: 1.2500e-04
Epoch 20/500
13/13 [==============================] - 0s 10ms/step - loss: 0.0469 - rmse: 0.0439 - val_loss: 0.2410 - val_rmse: 0.2380 - lr: 6.2500e-05
Epoch 21/500
13/13 [==============================] - 0s 10ms/step - loss: 0.0449 - rmse: 0.0420 - val_loss: 0.2425 - val_rmse: 0.2395 - lr: 6.2500e-05
Epoch 22/500
10/13 [====================>.........] - ETA: 0s - loss: 0.0413 - rmse: 0.0383
Epoch 22: ReduceLROnPlateau reducing learning rate to 3.125000148429535e-05.
13/13 [==============================] - 0s 10ms/step - loss: 0.0431 

13/13 [==============================] - 0s 11ms/step - loss: 0.0414 - rmse: 0.0385 - val_loss: 0.2696 - val_rmse: 0.2666 - lr: 1.9531e-06
Epoch 36/500
13/13 [==============================] - 0s 10ms/step - loss: 0.0475 - rmse: 0.0445 - val_loss: 0.2694 - val_rmse: 0.2665 - lr: 9.7656e-07
Epoch 37/500
13/13 [==============================] - 0s 10ms/step - loss: 0.0433 - rmse: 0.0404 - val_loss: 0.2692 - val_rmse: 0.2662 - lr: 9.7656e-07
Epoch 38/500
10/13 [====================>.........] - ETA: 0s - loss: 0.0476 - rmse: 0.0446
Epoch 38: ReduceLROnPlateau reducing learning rate to 4.882812731921149e-07.
13/13 [==============================] - 0s 10ms/step - loss: 0.0452 - rmse: 0.0422 - val_loss: 0.2692 - val_rmse: 0.2662 - lr: 9.7656e-07
Epoch 39/500
13/13 [==============================] - 0s 10ms/step - loss: 0.0468 - rmse: 0.0439 - val_loss: 0.2691 - val_rmse: 0.2662 - lr: 4.8828e-07
Epoch 40/500
13/13 [==============================] - 0s 10ms/step - loss: 0.0422 - rmse: 0.0393 

Epoch 21/500
10/13 [====================>.........] - ETA: 0s - loss: 0.0468 - rmse: 0.0439
Epoch 21: ReduceLROnPlateau reducing learning rate to 6.25000029685907e-05.
13/13 [==============================] - 0s 10ms/step - loss: 0.0456 - rmse: 0.0426 - val_loss: 0.2483 - val_rmse: 0.2453 - lr: 1.2500e-04
Epoch 22/500
13/13 [==============================] - 0s 10ms/step - loss: 0.0451 - rmse: 0.0422 - val_loss: 0.2475 - val_rmse: 0.2446 - lr: 6.2500e-05
Epoch 23/500
13/13 [==============================] - 0s 10ms/step - loss: 0.0434 - rmse: 0.0405 - val_loss: 0.2456 - val_rmse: 0.2427 - lr: 6.2500e-05
Epoch 24/500
 9/13 [==================>...........] - ETA: 0s - loss: 0.0434 - rmse: 0.0404
Epoch 24: ReduceLROnPlateau reducing learning rate to 3.125000148429535e-05.
13/13 [==============================] - 0s 10ms/step - loss: 0.0451 - rmse: 0.0422 - val_loss: 0.2485 - val_rmse: 0.2456 - lr: 6.2500e-05
Epoch 25/500
13/13 [==============================] - 0s 11ms/step - loss: 0.0425

Epoch 61/500
13/13 [==============================] - 0s 10ms/step - loss: 0.0399 - rmse: 0.0370 - val_loss: 0.2483 - val_rmse: 0.2453 - lr: 1.0000e-08
Epoch 62/500
13/13 [==============================] - 0s 10ms/step - loss: 0.0366 - rmse: 0.0337 - val_loss: 0.2483 - val_rmse: 0.2453 - lr: 1.0000e-08
Epoch 63/500
13/13 [==============================] - 0s 10ms/step - loss: 0.0370 - rmse: 0.0341 - val_loss: 0.2483 - val_rmse: 0.2453 - lr: 1.0000e-08
Epoch 64/500
13/13 [==============================] - 0s 10ms/step - loss: 0.0382 - rmse: 0.0352 - val_loss: 0.2483 - val_rmse: 0.2453 - lr: 1.0000e-08
Epoch 65/500
13/13 [==============================] - 0s 10ms/step - loss: 0.0368 - rmse: 0.0339 - val_loss: 0.2483 - val_rmse: 0.2453 - lr: 1.0000e-08
Epoch 65: early stopping
4
Epoch 1/500
13/13 [==============================] - 1s 19ms/step - loss: 0.4205 - rmse: 0.4177 - val_loss: 0.4433 - val_rmse: 0.4406 - lr: 0.0010
Epoch 2/500
13/13 [==============================] - 0s 11ms/step 

Epoch 40/500
13/13 [==============================] - 0s 11ms/step - loss: 0.0541 - rmse: 0.0511 - val_loss: 0.2503 - val_rmse: 0.2472 - lr: 9.7656e-07
Epoch 41/500
 9/13 [==================>...........] - ETA: 0s - loss: 0.0527 - rmse: 0.0497
Epoch 41: ReduceLROnPlateau reducing learning rate to 4.882812731921149e-07.
13/13 [==============================] - 0s 10ms/step - loss: 0.0551 - rmse: 0.0521 - val_loss: 0.2501 - val_rmse: 0.2471 - lr: 9.7656e-07
Epoch 42/500
13/13 [==============================] - 0s 10ms/step - loss: 0.0534 - rmse: 0.0504 - val_loss: 0.2501 - val_rmse: 0.2470 - lr: 4.8828e-07
Epoch 43/500
13/13 [==============================] - 0s 11ms/step - loss: 0.0565 - rmse: 0.0535 - val_loss: 0.2500 - val_rmse: 0.2470 - lr: 4.8828e-07
Epoch 44/500
 9/13 [==================>...........] - ETA: 0s - loss: 0.0538 - rmse: 0.0508
Epoch 44: ReduceLROnPlateau reducing learning rate to 2.4414063659605745e-07.
13/13 [==============================] - 0s 11ms/step - loss: 0.05

In [15]:
root = 'SI_Benchmarking/LR/'
if not os.path.exists(root):
    os.makedirs(root)

for i in range(5):
    pd.DataFrame(PP.ReMMNorm(loc.values, test_preds[i])).to_csv('SI_Benchmarking/LR/pred_' + str(i) + '.csv', header = None)

# NovoSpaRc

In [16]:
root = 'SI_Benchmarking/novosparc/'
if not os.path.exists(root):
    os.makedirs(root)

sta = time.time()

for i in range(5):
    
    tissue_with_markers = novosparc.cm.Tissue(dataset=adata[test_indices[i], :],\
                                              locations=Y[train_indices[i]],\
                                              atlas_matrix=adata[train_indices[i], :].X)
    
    gene_names = tissue_with_markers.gene_names
    markers_in_sc = np.array([], dtype='int')
    insitu_genes = adata.var_names.tolist()
                        
    for marker in insitu_genes:
        marker_index = np.where(gene_names == marker)[0]
        if len(marker_index) > 0:
            markers_in_sc = np.append(markers_in_sc, marker_index[0])
                                                                        
    tissue_with_markers.setup_reconstruction(markers_to_use=markers_in_sc)
    tissue_with_markers.reconstruct(alpha_linear=0.5, epsilon=5e-3)
    
    gw = tissue_with_markers.gw
    pd.DataFrame(np.matmul(gw, loc.values[train_indices[i]]) * gw.shape[0]).to_csv('SI_Benchmarking/novosparc/pred_' + str(i) + '.csv', header = None)
                                                                                            
end = time.time()
print((end - sta) / 60.0, 'min')

Setting up for reconstruction ... done ( 0.07 seconds )
Reconstructing spatial information with 14151 markers: 56 cells and 222 locations ... 
Trying with epsilon: 5.00e-03
Setting up for reconstruction ... done ( 0.04 seconds )
Reconstructing spatial information with 14151 markers: 56 cells and 222 locations ... 
Trying with epsilon: 5.00e-03
Setting up for reconstruction ... done ( 0.04 seconds )
Reconstructing spatial information with 14151 markers: 56 cells and 222 locations ... 
Trying with epsilon: 5.00e-03
Setting up for reconstruction ... done ( 0.04 seconds )
Reconstructing spatial information with 14151 markers: 55 cells and 223 locations ... 
Trying with epsilon: 5.00e-03
Setting up for reconstruction ... done ( 0.04 seconds )
Reconstructing spatial information with 14151 markers: 55 cells and 223 locations ... 
Trying with epsilon: 5.00e-03
0.22574344476064045 min


# SpaOTsc

In [17]:
def corr2_coeff(A, B):
    # Rowwise mean of input arrays & subtract from input arrays themeselves
    A_mA = A - A.mean(1)[:, None]
    B_mB = B - B.mean(1)[:, None]

    # Sum of squares across rows
    ssA = (A_mA**2).sum(1)
    ssB = (B_mB**2).sum(1)

    # Finally get corr coeff
    return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))

In [18]:
root = 'SI_Benchmarking/SpaOTsc/'
if not os.path.exists(root):
    os.makedirs(root)

sta = time.time()

coords = Y
is_dmat_all = np.zeros((coords.shape[0], coords.shape[0]))
for j in range(coords.shape[0]):
    is_dmat_all[j] = (((coords[j] - coords)**2).sum(axis = 1) ** 0.5)

for i in range(5):
    
    print(i)
    
    df_sc = pd.DataFrame(adata[test_indices[i],].X.toarray(),
                         columns = adata[test_indices[i],].var_names.tolist(),
                         index = adata[test_indices[i],].obs_names.tolist())
    df_is = pd.DataFrame(adata[train_indices[i],].X.toarray(),
                         columns = adata[train_indices[i],].var_names.tolist(),
                         index = adata[train_indices[i],].obs_names.tolist())

    sc_pcc = np.corrcoef(df_sc)
    
    mcc = corr2_coeff(df_sc.values, df_is.values)
    C = np.exp(1-mcc)
    
    is_dmat = is_dmat_all[train_indices[i]][:, train_indices[i]]
        
    issc = SpaOTsc.spatial_sc(sc_data=df_sc, is_data=df_is,
                              sc_dmat = np.exp(1-sc_pcc),
                              is_dmat = is_dmat)
    
    mapping = issc.transport_plan(C**2, alpha=0.1, rho=100.0, epsilon=1.0, cor_matrix=mcc, scaling=False)
    
    pd.DataFrame(np.matmul(mapping, loc.values[train_indices[i]]) * mapping.shape[0]).to_csv('SI_Benchmarking/SpaOTsc/pred_' + str(i) + '.csv', header = None)
    
end = time.time()
print((end - sta) / 60.0, 'min')


0
1
2
3
4
0.09932691256205241 min
